# Lab 1b - Getting started with python

In the first lab, we learned how to use ArcGIS to make a simple map, given shapefiles representing features of interest in the Oslo area. In today's lab, we're going to repeat the exercise, this time using `cartopy`, `geopandas`, and `matplotlib`. 

## Objectives
- become familiar with geopandas, cartopy, and matplotlib, including reading the provided documentation

## 1. Getting started
First, run the cell below. It will load the python modules we'll be using in today's lab, as well as define a few helper functions that we'll use later on. For now, don't worry too much about what each individual line does - we'll go over these in a bit more depth as we go. Remember also that if you get stuck, you can get help in a few ways:

1. the built-in help (i.e., `help(plt.text)`)
2. using ipython's help shortcut (i.e., `plt.text?`)
3. finding the online documentation for the module (usually achieved via option 4)
4. searching google (or your search engine of choice) 
5. consulting your favorite medicine man/shaman/spiritual guide
6. asking the instructor, who will in all likelihood resort to one of the other options (usually 5 or 4).

In [ ]:
%matplotlib notebook  # this lets us use the figures interactively

import geopandas as gpd
import cartopy
import matplotlib
import matplotlib.pyplot as plt
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import numpy as np

plt.ion() # make the plotting interactive

# generate matplotlib handles to create a legend of the features we put in our map.
def generate_handles(labels, colors):
    lc = len(colors)  # get the length of the color list
    handles = []
    for i in range(len(labels)):
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[i % lc]))
    return handles

# return a degree, minute representation given decimal degree
def dms_label(tick):
    degree = np.floor(tick)
    minute = np.round((tick - degree) * 600) / 10  # round to nearest tenth
    return "{}$^\circ${}\'".format(int(degree), int(minute))

# create a scale bar of length 8 km in the upper right corner of the map
def scale_bar(ax, location=(0.92, 0.95)):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]

    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    plt.plot([sbx, sbx - 8000], [sby, sby], color='k', linewidth=9, transform=tmc)
    plt.plot([sbx, sbx - 4000], [sby, sby], color='k', linewidth=6, transform=tmc)
    plt.plot([sbx-4000, sbx - 8000], [sby, sby], color='w', linewidth=6, transform=tmc)

    plt.text(sbx-500, sby-800, '8 km', transform=tmc)
    plt.text(sbx-4500, sby-800, '4 km', transform=tmc)
    plt.text(sbx-8500, sby-800, '0 km', transform=tmc)

## 2. Loading the data
Great. Now that we've imported most of the modules we'll be needing, and defined a few helper functions, we can actually load our data. To load the shapefile data we used in the first lab, we're going to be using [GeoPandas](http://geopandas.org/), an open-source package designed to make working with geospatial data in python easier. GeoPandas is built off of Pandas, a powerful data analysis tool. If you're interesting in learning more about using python for data analysis of any flavor, I very much recommend getting to know these packages more.

In [ ]:
water = gpd.read_file('data_files/Water.shp')
munis = gpd.read_file('data_files/Municipalities.shp')

GeoPandas loads the data associated with a shapefile into a GeoDataFrame, a tabular data structure that always has a column describing a feature's geometry. Each line in the table corresponds to a feature in the shapefile, just like the attribute table you are familiar with from ArcGIS/QGIS. 

We'll work with GeoDataFrames more in a later lab, but for now see if you can figure out the total area of lakes in the `Water` dataset. I'll provide two hints to get you started:

1. GeoDataFrames can be subset using a conditional and a column in the GeoDataFrame. For example, to select all water bodies with a surface area above 100 square meters, you might type something like `water[water['AREA'] > 100]`. Note that this would return a GeoDataFrame, that you could select columns from.
2. The numerical columns of a GeoDataFrame (also called GeoSeries) have built-in operators such as __max__, __min__, __mean__, and so on. To get the mean perimeter of the features in the `Water` dataset, you could type something like `water['PERIMETER'].mean()`

That should be enough to get you started - ask for help if you get stuck.

In [ ]:
munis # run this to see what the geodataframe looks like.
# below, write a command (or series of commands) to calculate the total area of lakes in the water dataset.

## 3. Creating maps with matplotlib and cartopy

Now that we're more familiar with the dataset, we can start building our map. For this portion of the lab, we'll be mostly using [matplotlib](https://matplotlib.org/), a python package designed for making plots and graphs, and [cartopy](https://scitools.org.uk/cartopy/docs/latest/), a package designed for making maps and representing geopatial data. 

In [ ]:
myFig = plt.figure(figsize=(9,9))  # create a figure of size 9x9 (representing the page size in inches),

myCRS = ccrs.UTM(32)  # create a Universal Transverse Mercator reference system to transform our data.
# be sure to fill in XX above with the correct number for the area we're working in. 

ax = plt.axes(projection=ccrs.Mercator())  # finally, create an axes object in the figure, using a Mercator
# projection, where we can actually plot our data.

### Adding data to the map

Now that we've created a figure and axes, we can start adding data to the map. To start, we'll add the municipal borders. 

In order to add these to the map, we first have to create features that we can add to the axes using the `ShapelyFeature` class from `cartopy.feature`. The initialization method for this class takes a minimum of two arguments, an __iterable__ containing the geometries that we're using, and a CRS representation. To add the municipal borders, then, we would use `munis['geometry']`, the GeoSeries of the feature geometries in our Municipalities shapefile, and `myCRS`, the CRS object representing the UTM Zone for the Oslo area.

The other arguments that we pass to `ShapelyFeature` tell `matplotlib` how to draw the features - in this case, with an edge color of black and a face color of gray. Once we've created the features, we add them to the axes using the `add_feature` method.

We'll also want to zoom the map into our area of interest using the boundary of the shapefile features (using `ax.set_extent`), and finally re-display the figure below so we don't have to scroll up and down all the time.

In [ ]:
# first, we just add the municipal boundaries, without giving different colors, using cartopy's ShapelyFeature
muni_features = ShapelyFeature(munis['geometry'], myCRS, edgecolor='k', facecolor='0.5')
ax.add_feature(muni_features) # add the features we've created to the map.

# using the boundary of the shapefile features, zoom the map to our area of interest
ax.set_extent(munis.total_bounds[[0, 2, 1, 3]], crs=myCRS) # because total_bounds gives output as xmin, ymin, xmax, ymax,
# but set_extent takes xmin, xmax, ymin, ymax, we re-order the coordinates here.

myFig # re-display the figure here.

This is fine, but a bit boring. For one thing, we might want to set different colors for the different municipalities, rather than having them all be the same color. To do this, we'll first have to count the number of unique municipalities in our dataset, then select colors to represent each of them. __Why might we do this, rather than just use the number of features in the dataset?__

Run the cell below to count the number of unique municipalities in the dataset, using the `unique` method on the __NAVN__ GeoSeries. Note that in addition to the standard indexing (i.e., `munis['NAVN']`), we are accessing __NAVN__ directly as an attribute of `munis` (i.e., `munis.NAVN`). Provided that the column name follows particular rules ([more on this here](http://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access)), there is no difference between these two methods - they give the same results.

In [ ]:
# get the number of unique municipalities we have in the dataset
num_munis = len(munis.NAVN.unique())
print('Number of unique features: {}'.format(num_munis)) # note how we're using {} and format here!

Now that you've found the number of colors you need to choose, you can use the image below to make a list of the colors. There are other ways to select colors using matplotlib, including using RGB values, but that's for another day. If you're interested in learning more, you can check out the documentation [here](https://matplotlib.org/api/colors_api.html).

![title](imgs/named_colors.png)
([source](https://matplotlib.org/2.0.2/examples/color/named_colors.html))

In [ ]:
# pick colors, add features to the map
muni_colors = []

# next, add the municipal outlines to the map using the colors that we've picked.
# here, we're iterating over the unique values in the 'NAVN' field.
# we're also setting the edge color to be black, with a line width of 0.5 pt. 
# Feel free to experiment with different colors and line widths.
for i, navn in enumerate(munis.NAVN.unique()):
    feat = ShapelyFeature(munis['geometry'][munis['NAVN'] == navn], myCRS, 
                          edgecolor='k',
                          facecolor=muni_colors[i],
                          linewidth=0.5)
    ax.add_feature(feat)
    
myFig # to show the updated figure

Now that we've done this for the municipal boundaries, we can also do this for the water dataset. Here, we want to have a different color for each of the water body types present in the dataset (innsjø, elv/bekk, and hav flate). Using the chart above, choose colors to represent these, and run the cell below.

In [ ]:
# we have water body types, innsjø (lake), elv/bekk (river/stream), and hav flate (ocean)
# choose three colors to represent the water bodies
water_colors = []

# here, we're setting the edge color to be the same as the face color. Feel free to change this around,
# and experiment with different line widths.
for i, w in enumerate(water.OBJTYPE.unique()):
    feat = ShapelyFeature(water['geometry'][water['OBJTYPE'] == w], myCRS, 
                          edgecolor=water_colors[i], 
                          facecolor=water_colors[i],
                          linewidth=0.3)
    ax.add_feature(feat)
    
myFig # to show the updated figure

### Adding labels and legends

Now that we have different colors for each of the municipal boundaries, and the different kinds of water bodies, it might be good to have a legend to keep everything straight. As we have two categories of data (municipal boundaries, water bodies), we might want to have two different groupings within our legend. To do this, we'll have to create our legend in stages. We'll start with the municipal boundaries.

First, we get handles for each of the municipal names, using the colors we defined earlier. Here, we're using our helper function `generate_handles`, which returns a list of `matplotlib` handles given a list of labels and colors. We can pass the result of this to `plt.legend`, to generate a legend for the municipal boundaries data. Feel free to experiment with the placement (by changing __loc__ and/or __bbox_to_anchor__), or the font size, the title font size, and so on. 

Once you're satisfied, be sure to run the next cell (`ax.add_artis(leg1)`), which will permanently add the legend to our plot. If we don't do this step, `matplotlib` will overwrite our municipalities plot with the water bodies plot, when we add that legend to the plot.

In [ ]:
muni_handles = generate_handles(munis.NAVN.unique(), muni_colors)

leg1 = plt.legend(muni_handles, munis.NAVN.unique(), title='Municipalities', title_fontsize=14, 
                  fontsize=12, loc='lower left', bbox_to_anchor=(-0.295, 0), frameon=False)

myFig # to show the updated figure

In [ ]:
ax.add_artist(leg1)

Now, add the water bodies legend to the plot. Feel free to experiment with the location, font sizes, etc., as you did before.

In [ ]:
water_handles = generate_handles(water.OBJTYPE.unique(), water_colors)

plt.legend(water_handles, water.OBJTYPE.unique(), title='Water Type', title_fontsize=14,
           fontsize=12, loc='lower left', bbox_to_anchor=(-0.295, 0.36), frameon=False)

myFig # to show the updated figure

Now that we have legends, let's go ahead and add grid lines to our plot. I've chosen the same gridlines as we used in the previous lab, though feel free to change this. What happens if you delete the first and/or last value from xlocs and ylocs? Can you change the labels to show only on the bottom and left side of the map?

In [ ]:
gridlines = ax.gridlines(draw_labels=True,
                         xlocs=[10.25, 10.416667, 10.5, 10.5833333, 10.6666667, 10.75], 
                         ylocs=[59.5, 59.8333333, 59.9166666667, 60.2])
gridlines.ylabels_left = False
gridlines.xlabels_bottom = False
ax.set_extent(water.total_bounds[[0, 2, 1, 3]], crs=myCRS)
myFig # to show the updated figure

These are fine, but maybe we want to display the tick labels as (degrees, minutes) rather than decimal degrees. To do this, we can run the following cell, which will find all Text objects within our axes. Then, if they aren't blank, we run the output of `float(Text.get_text())` on them through our helper function `dms_label`, which converts the coordinate to the form __degrees° minutes'__, and change the font size to 10. 

Feel free to try a different font size, though note that passing the output of `get_text()` to `float()` might not work once we've converted the characters. How can you get around this problem?

Finally, try running ax.get_children() in a blank cell, to see all of the different objects present in our (relatively) simple map - think about how you might figure out what each of these objects are or what they represent?

In [ ]:
text_list = [c for c in ax.get_children() if isinstance(c, matplotlib.text.Text)]
for tl in text_list:
    if tl.get_text() != '':
        tl.set_text(dms_label(float(tl.get_text())))
        tl.set_fontsize(10)
myFig

Excellent. Now, let's add text labels for each of our individual municipalities. One problem that we have, though, is that we have multiple features representing one municipality - we probably don't want to label them all. What we can do, then, is join the features representing each municipality into a `MultiPolygon`, and compute the __centroid__ of that object. Then, we can add our text with the municipality's name at that location. Look over the cell below, and make sure you understand what it's doing. Feel free to discuss with your neighbor or the instructor if you're not sure.

In [ ]:
from shapely.geometry import MultiPolygon

for navn in munis.NAVN.unique():
    subframe = munis[munis['NAVN'] == navn]
    mpoly_list = []
    for geom in subframe['geometry']:
        mpoly_list.append(geom)
    mpoly = MultiPolygon(mpoly_list)
    
    x, y = mpoly.centroid.x, mpoly.centroid.y
    plt.text(x, y, navn, fontsize=14, transform=myCRS)

Last but not least, let's add a scale bar to the plot. The scale_bar function we've defined above will produce a scale bar with divisions at 4 and 8 km, with a location in the upper right corner as default. Try to experiment with this a bit - can you design a scale bar with divisions at 1, 2, and 4 km? It's not as straightforward as it is in ArcGIS, but it might provide an interesting challenge if you're interested in developing your programming skills a bit.

In [ ]:
scale_bar(ax)

Finally, we'll save our figure. The command written below will save the figure to the current folder, in a file called `map.png`, with no border around the outside of the map, and with a resolution of 300 dots per inch. As always, feel free to change these parameters.

In [ ]:
plt.savefig('map.png', bbox_inches='tight', dpi=300)